In [1]:
import kuzu
import shutil

In [2]:
shutil.rmtree("./knowledgebase", ignore_errors=True)
db = kuzu.Database('./knowledgebase', buffer_pool_size=1024**3)
conn = kuzu.Connection(db)

In [3]:
#Define KictStat Knowledge Base Schema
conn.execute("CREATE NODE TABLE MatchNode(match_id INT64, match_date DATE, kick_off TIMESTAMP, home_score INT64, away_score INT64, match_status STRING, match_status_360 STRING, match_week INT64, PRIMARY KEY (match_id))")
conn.execute("CREATE NODE TABLE Competition(competition_id INT64, country_name STRING, competition_name STRING, PRIMARY KEY (competition_id))")
conn.execute("CREATE NODE TABLE Season(season_id INT64,season_name STRING, PRIMARY KEY (season_id))")
conn.execute("CREATE NODE TABLE Stadium(id INT64, name STRING, PRIMARY KEY (id))")
conn.execute("CREATE NODE TABLE Country(id INT64, name STRING, PRIMARY KEY (id))") 
conn.execute("CREATE NODE TABLE Referee(id INT64, name STRING, PRIMARY KEY (id))")
conn.execute("CREATE NODE TABLE Managers(id INT64, name STRING, nickname STRING, dob DATE, PRIMARY KEY (id))")
conn.execute("CREATE NODE TABLE CompetitionStage(id INT64, name STRING, PRIMARY KEY (id))")
conn.execute("CREATE NODE TABLE Team(team_id INT64, team_name STRING, team_gender STRING, PRIMARY KEY (team_id))") 
conn.execute("CREATE REL TABLE MatchHomeTeam(FROM MatchNode TO Team, HomeTeamGroup STRING)")
conn.execute("CREATE REL TABLE MatchAwayTeam(FROM MatchNode TO Team, AwayTeamGroup STRING)")
conn.execute("CREATE REL TABLE TeamCountry(FROM Team TO Country)")
conn.execute("CREATE REL TABLE TeamManagers(FROM Team TO Managers)")
conn.execute("CREATE REL TABLE MatchPartOfCompetition(FROM MatchNode TO Competition)")
conn.execute("CREATE REL TABLE MatchHeldInSeason(FROM MatchNode TO Season)")
conn.execute("CREATE REL TABLE StadiumCountry(FROM Stadium TO Country)")
conn.execute("CREATE REL TABLE RefereeCountry(FROM Referee TO Country)")
conn.execute("CREATE REL TABLE ManagerCountry(FROM Managers TO Country)")
conn.execute("CREATE REL TABLE MatchCompetitionStage(FROM MatchNode TO CompetitionStage)") 
conn.execute("CREATE REL TABLE MatchHeldInStadium(FROM MatchNode TO Stadium)")
conn.execute("CREATE REL TABLE MatchReferee(FROM MatchNode TO Referee)")

conn.execute("CREATE NODE TABLE Player(player_id INT64, player_name STRING, player_nickname STRING, PRIMARY KEY (player_id))")
conn.execute("CREATE REL TABLE MatchPlayerPositions(FROM MatchNode TO Player, position_id INT64, position STRING, from_time STRING, to_time STRING, from_period INT64, to_period INT64, start_reason STRING, end_reason STRING)")
conn.execute("CREATE REL TABLE MatchPlayers(FROM MatchNode TO Player, jersey_number INT64)")
conn.execute("CREATE REL TABLE MatchPlayerCards(FROM MatchNode TO Player, time STRING, card_type STRING, reason STRING, period INT64)")
conn.execute("CREATE REL TABLE PlayerCountry(FROM Player TO Country)")
conn.execute("CREATE REL TABLE PlayerTeam(FROM Player TO Team)")

conn.execute("CREATE NODE TABLE Event(event_id STRING, index INT64, period INT64, timestamp STRING, minute INT64, second INT64, type_id INT64, type_name STRING, possession INT64, play_pattern_id INT64, play_pattern_name STRING, position_id INT64, position_name STRING, location STRING, duration FLOAT, under_pressure BOOLEAN, off_camera BOOLEAN, out BOOLEAN, related_events STRING, bad_behaviour_card_id INT64, bad_behaviour_card_name STRING, PRIMARY KEY (event_id))")
conn.execute("CREATE REL TABLE EventRelatedToPlayer(FROM Event TO Player)")
conn.execute("CREATE REL TABLE EventRelatedToTeam(FROM Event TO Team)")
conn.execute("CREATE REL TABLE EventPossessionTeam(FROM Event TO Team)")

In [4]:
#Populate knowledge base
conn.execute('COPY MatchNode FROM "data/matches.csv"')
conn.execute('COPY Competition FROM "data/competition.csv"')
conn.execute('COPY Season FROM "data/season.csv"')
conn.execute('COPY Stadium FROM "data/stadium.csv"')
conn.execute('COPY Country FROM "data/country.csv"') 
conn.execute('COPY Referee FROM "data/referee.csv"')
conn.execute('COPY Managers FROM "data/managers.csv"')
conn.execute('COPY CompetitionStage FROM "data/competition_stage.csv"')
conn.execute('COPY Team FROM "data/teams.csv"')
conn.execute('COPY TeamCountry FROM "data/teams_country.csv"') 
conn.execute('COPY TeamManagers FROM "data/teams_managers.csv"') 
conn.execute('COPY MatchHomeTeam FROM "data/match_home_team.csv"') 
conn.execute('COPY MatchAwayTeam FROM "data/match_away_team.csv"')
conn.execute('COPY MatchPartOfCompetition FROM "data/match_part_of_competition.csv"')
conn.execute('COPY MatchHeldInSeason FROM "data/match_held_in_season.csv"')
conn.execute('COPY MatchHeldInStadium FROM "data/match_held_in_stadium.csv"')
conn.execute('COPY StadiumCountry FROM "data/stadium_country.csv"')
conn.execute('COPY RefereeCountry FROM "data/referee_country.csv"')
conn.execute('COPY ManagerCountry FROM "data/manager_country.csv"')
conn.execute('COPY MatchCompetitionStage FROM "data/match_competition_stage.csv"') 
conn.execute('COPY MatchReferee FROM "data/match_referee.csv"')

conn.execute('COPY Player FROM "data/player.csv"')
conn.execute('COPY MatchPlayerPositions FROM "data/match_player_positions.csv"')
conn.execute('COPY MatchPlayers FROM "data/match_player.csv"')
conn.execute('COPY MatchPlayerCards FROM "data/match_player_cards.csv"')  
conn.execute('COPY PlayerCountry FROM "data/player_country.csv"')
conn.execute('COPY PlayerTeam FROM "data/player_team.csv"')

conn.execute('COPY Event FROM "data/event.csv"')
conn.execute('COPY EventRelatedToPlayer FROM "data/event_related_to_player.csv"')
conn.execute('COPY EventRelatedToTeam FROM "data/event_relates_to_team.csv"')
conn.execute('COPY EventPossessionTeam FROM "data/possession_team.csv"')

In [5]:
# #Execute a simple query:
# results = conn.execute('MATCH (u:Competition) RETURN u.competition_id, u.country_name, u.competition_name;')
# while results.has_next():
#     print(results.get_next())

[11, 'Spain', 'La Liga']
[116, 'North and Central America', 'North American League']
[12, 'Italy', 'Serie A']
[1238, 'India', 'Indian Super league']
[1470, 'International', 'FIFA U20 World Cup']
[16, 'Europe', 'Champions League']
[2, 'England', 'Premier League']
[35, 'Europe', 'UEFA Europa League']
[37, 'England', "FA Women's Super League"]
[43, 'International', 'FIFA World Cup']
[49, 'United States of America', 'NWSL']
[53, 'Europe', "UEFA Women's Euro"]
[55, 'Europe', 'UEFA Euro']
[7, 'France', 'Ligue 1']
[72, 'International', "Women's World Cup"]
[81, 'Argentina', 'Liga Profesional']
[87, 'Spain', 'Copa del Rey']
[9, 'Germany', '1. Bundesliga']


In [6]:
def execute_and_print_result(query):
  results = conn.execute(query)
  while results.has_next():
      print(results.get_next())
  results.close()

In [7]:
def execute_and_return_result(query):
  results = conn.execute(query)
  return_values = []
  while results.has_next():
      return_values.append(results.get_next())
  results.close()
  return return_values

In [9]:
def sample_results():
    #Query1 answer
    execute_and_print_result('MATCH (a:MatchNode)-[e:MatchPlayers]->(b:Player) WHERE b.player_name = "Glenn Murray" AND EXISTS \
    {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="2015/2016"} AND EXISTS \
    {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "Premier League"} RETURN count(a.match_id)') 

    #Query2 answer - sum both scores
    execute_and_print_result('MATCH (a:MatchNode)-[e:MatchAwayTeam]->(t:Team) WHERE t.team_name = "Leicester City" AND EXISTS \
    {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="2015/2016"} AND EXISTS \
    {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "Premier League"} RETURN sum(a.home_score)')
    execute_and_print_result('MATCH (a:MatchNode)-[e:MatchHomeTeam]->(t:Team) WHERE t.team_name = "Leicester City" AND EXISTS \
    {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="2015/2016"} AND EXISTS \
    {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "Premier League"} RETURN sum(a.away_score)')

    #Query3 answer - ouputs [goals score, goals conceded] for each match in the given stadium
    execute_and_print_result('MATCH (a:MatchNode)-[MatchHeldInStadium]->(d:Stadium) WHERE d.name = "King Power Stadium " AND EXISTS \
    {MATCH (a)-[MatchAwayTeam]->(t:Team) WHERE t.team_name = "Leicester City"} AND EXISTS \
    {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="2015/2016"} AND EXISTS \
    {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "Premier League"} RETURN a.away_score, a.home_score')

    execute_and_print_result('MATCH (a:MatchNode)-[MatchHeldInStadium]->(d:Stadium) WHERE d.name = "King Power Stadium " AND EXISTS \
    {MATCH (a)-[MatchHomeTeam]->(t:Team) WHERE t.team_name = "Leicester City"} AND EXISTS \
    {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="2015/2016"} AND EXISTS \
    {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "Premier League"} RETURN a.home_score, a.away_score')

    #Query4 answer
    execute_and_print_result('MATCH (a:MatchNode)-[MatchReferee]->(r:Referee) WHERE r.name <>"" AND EXISTS \
    {MATCH (a)-[e:MatchPlayers]->(b:Player) WHERE b.player_name = "Glenn Murray"} AND EXISTS \
    {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="2015/2016"} AND EXISTS \
    {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "Premier League"} RETURN DISTINCT r.name') 

    #Query5 answer
    execute_and_print_result('MATCH (t:Team)-[TeamManagers]->(m:Managers) WHERE m.name <> "" AND EXISTS \
    {MATCH (m)-[ManagerCountry]->(c:Country) WHERE c.name = "Germany" }  AND EXISTS \
    {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="2015/2016"} AND EXISTS \
    {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "Premier League"}RETURN DISTINCT t.team_name') 

    #Query6 answer
    execute_and_print_result('MATCH (t:Team)-[TeamManagers]->(m: Managers) WHERE m.name <> "" AND EXISTS \
    {MATCH (t)<-[PlayerTeam]-(a:Player) WHERE a.player_name = "Glenn Murray"}  RETURN DISTINCT m.name') 

    #Query7 answer
    execute_and_print_result('MATCH (a:Player)<-[EventRelatedToPlayer]-(e:Event) WHERE e.event_id <> "" AND EXISTS \
    {MATCH (a)<-[MatchPlayers]-(MatchNode) WHERE a.player_name = "So-Yun Ji"}  AND EXISTS \
    {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="2015/2016"} AND EXISTS \
    {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "Premier League"} RETURN count(e.event_id)') 

    #Query8 answer
    execute_and_print_result('MATCH (a:Player)<-[EventRelatedToPlayer]-(e:Event) WHERE e.type_name = "Own Goal For" AND EXISTS \
    {MATCH (a)<-[MatchPlayers]-(MatchNode) WHERE a.player_name = "So-Yun Ji"} AND EXISTS \
    {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="2015/2016"} AND EXISTS \
    {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "Premier League"} RETURN count(e.event_id)') 

    #Query9 answer
    execute_and_print_result('MATCH (a:Player)<-[EventRelatedToPlayer]-(e:Event) WHERE \
    e.bad_behaviour_card_name = "Yellow Card" OR e.bad_behaviour_card_name = "Second Yellow" OR e.bad_behaviour_card_name = "Red Card" AND EXISTS \
    {MATCH (a)<-[MatchPlayers]-(MatchNode) WHERE a.player_name = "So-Yun Ji"}  AND EXISTS \
    {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="2015/2016"} AND EXISTS \
    {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "Premier League"} RETURN count(e.event_id)') 

    #Query10 answer
    execute_and_print_result('MATCH (a:Player)<-[EventRelatedToPlayer]-(e:Event) WHERE e.type_name = "Own Goal Against" AND EXISTS \
    {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="2015/2016"} AND EXISTS \
    {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "Premier League"} RETURN a.player_name') 

[18]
[18]


In [20]:
def rank_player_in_matches(competition_name, season_name, player_name):
    all_players = execute_and_return_result(
        'MATCH (a:MatchNode)-[e:MatchPlayers]->(b:Player) WHERE EXISTS \
        {MATCH (a)-[MatchHeldInSeason]->(s:Season) WHERE s.season_name="{}"} AND EXISTS \
        {MATCH (a)-[MatchPartOfCompetition]->(c:Competition) WHERE c.competition_name = "{}"} \
        RETURN b.player_name, count(a.match_id) ORDER BY count(a.match_id) DESC'.format(season_name, competition_name))

    rank = 1
    for p in all_players:
        if p[0] == player_name:
            return player_name, rank
        rank += 1
    return player_name, "Not Found"

In [21]:
# rank_player_in_matches("Premier League", "2015/2016", "Glenn Murray")

KeyError: 'MATCH (a)-'